In [ ]:
from api import NFL, CLEAN
import pyodbc
import os
import urllib
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import pandas as pd
from dotenv import load_dotenv
load_dotenv()
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", 100)

sql_server = "192.168.1.17\SQLEXPRESS"
database = "FantasySidelines"
sql_username = os.getenv("sql_username")
sql_password = os.getenv("sql_password")

connection_string = (r"DRIVER=ODBC Driver 17 for SQL Server;"
                     +fr"SERVER={sql_server};"
                     +fr"Database={database};"
                     +fr"UID={sql_username};"
                     +fr"PWD={sql_password};")

cxn = pyodbc.connect(connection_string)

connection_url = URL.create("mssql+pyodbc", 
                            query={"odbc_connect": connection_string})
engine = create_engine(connection_url)
conn = engine.connect()

api_key = [os.getenv("sportradar_apikey"), os.getenv("sportradar_apikey2"), os.getenv("sportradar_apikey3")]
year = [2020, 2019, 2018, 2017, 2016, 2015]

In [ ]:
def get_season(api_key, year, access_level="trial", version="v7", language_code="en", format_="json", sleep_time=2, timeout=5):
    season = NFL(api_key, access_level=access_level, version=version, language_code=language_code, format_=format_, sleep_time=sleep_time, timeout=timeout)

    schedule_api = season.season_schedule(year)
    schedule = CLEAN().schedule_clean(schedule_api)
    game_id_list = list(schedule["game.id"].unique())
    
    players_stats = pd.DataFrame()
    teams_stats = pd.DataFrame()

    for game_id in range(len(game_id_list)):
        statistics_api = season.game_statistics(game_id_list[game_id])
        
        players = CLEAN().player_stats_clean(statistics_api)
        teams = CLEAN().teams_stats_clean(statistics_api)
        
        players_stats = pd.concat([players, players_stats])
        teams_stats = pd.concat([teams, teams_stats])

    return schedule, players_stats, teams_stats

In [ ]:
# schedule_2020, players_stats_2020, teams_stats_2020 = get_season(api_key[0], year[0])
# schedule_2019, players_stats_2019, teams_stats_2019 = get_season(api_key[0], year[1])
# schedule_2018, players_stats_2018, teams_stats_2018 = get_season(api_key[0], year[2])
# schedule_2017, players_stats_2017, teams_stats_2017 = get_season(api_key[2], year[3])
# schedule_2016, players_stats_2016, teams_stats_2016 = get_season(api_key[1], year[4])
# schedule_2015, players_stats_2015, teams_stats_2015 = get_season(api_key[1], year[5])
# schedule = pd.concat([schedule_2020, schedule_2019, schedule_2018, schedule_2017, schedule_2016, schedule_2015])
# players_stats = pd.concat([players_stats_2020, players_stats_2019, players_stats_2018, players_stats_2017, players_stats_2016, players_stats_2015])
# teams_stats = pd.concat([teams_stats_2020, teams_stats_2019, teams_stats_2018, teams_stats_2017, teams_stats_2016, teams_stats_2015])

In [ ]:
schedule.to_sql("schedule", engine, if_exists="append", index=False, schema="dbo" chunksize=500)
players_stats.to_sql("playerStats", engine, if_exists="append", index=False, schema="dbo", chunksize=500)
teams_stats.to_sql("teamStats", engine, if_exists="append", index=False, schema="dbo", chunksize=500)

In [ ]:
# import smtplib
# import time

# email = os.getenv("notification_email")
# email_password = os.getenv("notification_password")
# error = ""
# try:
#     pass
# except Exception as _ex:
#     error = repr(_ex)
# to = "cuddebtj@gmail.com"    
# if error != "":
#     subject = "Python Script Error"
#     body = f"Error raised: {error}\nTime to error: {(end_time-start_time)/60}"
# else:
#     subject = "Python Script Complete"
#     body = f"Script complete without errors.\nTime to error: {(end_time-start_time)/60}"
# email_text = f"From: {email}" \
#              f"To: {to}"\
#              f"Subject: {subject}"\
#              f"{body}"
# server = smtplib.SMTP_SSL("smtp.gmail.com", 465)
# server. ehlo()
# server.login(email, email_password)
# server.sendmail(email, to, email_text)
# server.close()